# Neuronexus Analysis v6

**PLX → LFP Filtering → Spike Sorting → Phase Locking**

このノートブックでは、PLXファイルを読み込んでLFP解析・スパイクソーティング・位相ロック解析を順番に実行します。  
各セクションのON/OFFは下の設定セルで切り替えてください。

## 0. 設定

In [5]:
import os
os.add_dll_directory(r"c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\torch\lib")
import torch
print("PyTorch:", torch.__version__, "CUDA:", torch.cuda.is_available())
import sys
import numpy as np

# ============================================================
# 解析設定 - 必要に応じて変更してください
# ============================================================

# PLXファイルパス（空文字列 → GUIで選択）
PLX_FILE = r"\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\260114-4_Wildtype_S1BF_920_wisker\260114-4_Wildtype_S1BF_920_wisker.plx"

# 出力ディレクトリ（空文字列 → PLXファイルと同じフォルダ）
OUTPUT_DIR = r"\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\0209解析"

# 各ステップのON/OFF
RUN_LFP    = True    # LFP処理を実行するか
RUN_SPIKE  = True    # スパイクソーティングを実行するか
RUN_PHASE  = True    # 位相ロック解析を実行するか

# Widebandデータを読み込むか（メモリ節約のためFalseにできる）
LOAD_WIDEBAND = True

PyTorch: 2.10.0+cu126 CUDA: True


## 1. PLXファイル読み込み

In [6]:


# NumPy 2.0+ 互換パッチ（neo/quantities より先に実行）
import numpy_compat

# Matplotlib をノートブック用に設定
# GUIウィンドウを使うので %matplotlib tk を使用
%matplotlib tk

print("=" * 60)
print("  Neuronexus Analysis v6 (Notebook)")
print("=" * 60)

  Neuronexus Analysis v6 (Notebook)


In [7]:
# PLXファイル選択
plx_file = PLX_FILE
if not plx_file:
    from get_path import get_path
    plx_file = get_path(mode='file', file_type='plx', initial_dir=r'\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\260114-4_Wildtype_S1BF_920_wisker')

if not plx_file or not os.path.exists(plx_file):
    raise FileNotFoundError(f"PLXファイルが見つかりません: {plx_file}")

print(f"\n[Step 0] PLX読み込み: {os.path.basename(plx_file)}")
from data_loader import load_plx
plx_data = load_plx(plx_file, load_wideband=LOAD_WIDEBAND)

if OUTPUT_DIR:
    plx_data.output_dir = OUTPUT_DIR
output_dir = plx_data.output_dir
basename = plx_data.basename

# 結果を保持
lfp_results = None
spike_results = None
phase_results = None

print(f"\n出力先: {output_dir}")


[Step 0] PLX読み込み: 260114-4_Wildtype_S1BF_920_wisker.plx
[DataLoader] 読み込み: 260114-4_Wildtype_S1BF_920_wisker.plx


Parsing signal channels: 100%|██████████| 128/128 [00:00<00:00, 42659.59it/s]
Parsing spike channels: 16it [00:00, 16001.16it/s]
Parsing event channels: 100%|██████████| 43/43 [00:00<?, ?it/s]


  LFP: (112445, 2), fs=1000Hz
  LFP: (112460, 16), fs=1000Hz
  Wideband: (4498350, 16), fs=40000Hz
  フレーム同期: EVT03, 1241個
  刺激イベント: 90個
  動画: 1241 フレーム
  Trim: 7.71s ~ 106.89s
  スパイク: 16 trains (0 sorted)
[DataLoader] 完了: 112.5秒のデータ

出力先: \\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\0209解析


## 2. LFP Filtering

LFP Filter GUI が起動します。`plx_data` のファイルパス・チャンネル情報等は自動でGUIに反映されます。  
フィルタ設定を行い「実行」ボタンを押してください。GUI を閉じると次のセルに進めます。

In [8]:
if RUN_LFP:
    print("[Step 1] LFP Filter GUI を起動...")

    from lfp_filter_gui import LfpFilterGUI

    lfp_config_holder = [None]

    def on_lfp_done(config, _plx_data):
        lfp_config_holder[0] = config

    # plx_data を渡すと、ファイルパス・データ情報が自動でGUIに反映される
    gui = LfpFilterGUI(plx_data=plx_data, on_done=on_lfp_done)
    gui.run()

    lfp_config = lfp_config_holder[0]
    if lfp_config is None:
        print("  ⚠ LFP処理がキャンセルされました")
    else:
        print("[Step 1b] LFPパイプライン実行...")
        from lfp_pipeline import run_lfp_pipeline
        lfp_results = run_lfp_pipeline(lfp_config, plx_data)
        print("  ✓ LFP処理完了")
else:
    print("[Step 1] LFP処理: スキップ")

[Step 1] LFP Filter GUI を起動...
[LFP GUI] 設定読み込み: \\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\lfp_config.json
[Step 1b] LFPパイプライン実行...
=== LFPパイプライン開始: 260114-4_Wildtype_S1BF_920_wisker ===
  LFP: (112460, 16), fs=1000Hz
  Trim: 7.7~106.9s

[1/6] フィルタリング...
  バンドパス: 0.1-100.0Hz [FIR]

  高調波ノイズ除去...
  高調波ノイズ除去: [10.0, 30.0, 50.0, 70.0] Hz
  除去: ['10Hz', '30Hz', '50Hz', '70Hz']

[2/6] チャンネル処理...
    異常チャンネル: D7(Ch4), var=0.033510
  除外: [7], 残り: 15ch

[3/6] モーション解析...
  ROIを選択 (Enter確定, C取消)
  ノイズ区間: 28.2%

[4/6] ICA...
    IC9: ratio=16.50 → 除去
    IC1: ratio=12.88 → 除去
    IC4: ratio=9.32 → 除去
    IC8: ratio=4.50 → 除去
  除去: 4 成分

[5/6] パワー解析...
  刺激イベント: 90個

[6/6] プロット出力...

  CSV保存...

=== LFPパイプライン完了 ===
  ✓ LFP処理完了


## 3. Spike Sorting

Spike Sorting GUI が起動します。各チャンネルのスパイクソーティングを行ってください。

In [9]:
if RUN_SPIKE and plx_data.wideband_raw is not None:
    print("[Step 2] Spike Sorting GUI を起動...")

    from spike_sort_gui import SpikeSortGUI

    spike_holder = [None]

    def on_spike_done(results):
        spike_holder[0] = results

    gui = SpikeSortGUI(
        wideband_data=plx_data.wideband_raw,
        fs=plx_data.wideband_fs,
        output_dir=output_dir,
        basename=basename,
        on_done=on_spike_done
    )
    gui.run()
    spike_results = spike_holder[0]

    if spike_results:
        from spike_plotting import plot_spike_grand_summary, plot_quality_table
        plot_spike_grand_summary(spike_results, output_dir, basename,
                                 show=True, save=True)
        plot_quality_table(spike_results, output_dir, basename,
                           show=True, save=True)
        n_units = sum(len(r.units) for r in spike_results.values())
        print(f"  ✓ スパイクソーティング完了: {n_units} units")
    else:
        print("  ⚠ スパイクソーティングがキャンセルされました")

elif not RUN_SPIKE:
    print("[Step 2] スパイクソーティング: スキップ")
else:
    print("[Step 2] Widebandデータなし - スパイクソーティングをスキップ")

[Step 2] Spike Sorting GUI を起動...
=== スパイクソーティング ===
チャンネル: 16, fs: 40000 Hz
フィルタ: 300.0-3000.0 Hz

フィルタリング...

--- Channel 0 ---
  Ch0: 検出 781 スパイク
  Ch0: PCA寄与率 [0.33, 0.26, 0.18]


c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


  Ch0: 4 クラスター
    Unit1: n= 141, amp=-0.0553, SNR=2.5, ISI=3.6% ⚠
    Unit2: n= 220, amp=-0.0557, SNR=3.0, ISI=4.1% ⚠
    Unit3: n= 406, amp=-0.0522, SNR=4.4, ISI=0.0% ✓

--- Channel 1 ---
  Ch1: 検出 1048 スパイク
  Ch1: PCA寄与率 [0.29, 0.26, 0.16]
  Ch1: 3 クラスター
    Unit1: n= 377, amp=-0.0491, SNR=2.9, ISI=6.9% [MUA]
    Unit3: n= 643, amp=-0.0483, SNR=3.3, ISI=4.8% ⚠

--- Channel 2 ---
  Ch2: 検出 903 スパイク
  Ch2: PCA寄与率 [0.34, 0.25, 0.17]
  Ch2: 3 クラスター
    Unit1: n= 497, amp=-0.0503, SNR=3.7, ISI=1.0% ✓
    Unit2: n=  59, amp=-0.0587, SNR=1.7, ISI=8.6% [MUA]
    Unit3: n= 337, amp=-0.0487, SNR=3.5, ISI=6.8% [MUA]

--- Channel 3 ---
  Ch3: 検出 1079 スパイク
  Ch3: PCA寄与率 [0.32, 0.25, 0.18]
  Ch3: 3 クラスター
    Unit1: n= 769, amp=-0.0469, SNR=3.2, ISI=4.4% ⚠
    Unit2: n= 263, amp=-0.0476, SNR=3.2, ISI=6.1% [MUA]
    Unit3: n=  35, amp=-0.0637, SNR=1.6, ISI=5.9% [MUA]

--- Channel 4 ---
  Ch4: 検出 1075 スパイク
  Ch4: PCA寄与率 [0.31, 0.26, 0.15]
  Ch4: 3 クラスター
    Unit1: n=  20, amp=-0.0591, SNR=0.9, ISI=1

## 4. Phase Locking Analysis

位相ロック解析GUI が起動します。解析帯域や条件を設定してください。

In [10]:
if RUN_PHASE and lfp_results is not None:
    print("[Step 3] Phase Locking GUI を起動...")

    condition_masks = None
    if all(k in lfp_results for k in ['clean_baseline', 'clean_stim', 'clean_post']):
        condition_masks = {
            'baseline': lfp_results['clean_baseline'],
            'stim': lfp_results['clean_stim'],
            'post': lfp_results['clean_post']
        }

    from phase_gui import PhaseGUI

    phase_holder = [None, None]

    def on_phase_done(pr, cr):
        phase_holder[0] = pr
        phase_holder[1] = cr

    gui = PhaseGUI(
        lfp_cleaned=lfp_results.get('lfp_cleaned'),
        lfp_times=lfp_results.get('lfp_times'),
        fs=lfp_results.get('fs'),
        segment=plx_data.segment,
        spike_results=spike_results,
        stim_times=lfp_results.get('stim_times'),
        condition_masks=condition_masks,
        original_ch_numbers=lfp_results.get('original_ch_numbers', []),
        output_dir=output_dir,
        basename=basename,
        on_done=on_phase_done
    )
    gui.run()
    phase_results = phase_holder[0]

    if phase_results:
        print("  ✓ 位相ロック解析完了")
    else:
        print("  ⚠ 位相ロック解析がキャンセルされました")

elif not RUN_PHASE:
    print("[Step 3] 位相ロック解析: スキップ")
else:
    print("[Step 3] LFPデータなし - 位相ロック解析をスキップ")

[Step 3] Phase Locking GUI を起動...
  ユニット: ch0_unit1 (141 spikes)
  delta (1.0-4.0 Hz)...
    Ch0: MRL=0.122, PPC=0.008, p=0.1323, n=136
    Ch1: MRL=0.190, PPC=0.029, p=0.0074*, n=136
    Ch2: MRL=0.224, PPC=0.043, p=0.0011*, n=136
    Ch3: MRL=0.233, PPC=0.048, p=0.0006*, n=136
    Ch4: MRL=0.218, PPC=0.040, p=0.0016*, n=136
    Ch5: MRL=0.218, PPC=0.040, p=0.0016*, n=136
    Ch6: MRL=0.219, PPC=0.041, p=0.0015*, n=136
    Ch7: MRL=0.172, PPC=0.022, p=0.0178*, n=136
    Ch8: MRL=0.183, PPC=0.026, p=0.0107*, n=136
    Ch9: MRL=0.168, PPC=0.021, p=0.0216*, n=136
    Ch10: MRL=0.108, PPC=0.004, p=0.2047, n=136
    Ch11: MRL=0.052, PPC=-0.005, p=0.6916, n=136
    Ch12: MRL=0.056, PPC=-0.004, p=0.6554, n=136
    Ch13: MRL=0.056, PPC=-0.004, p=0.6533, n=136
    Ch14: MRL=0.040, PPC=-0.006, p=0.8048, n=136
  theta (4.0-8.0 Hz)...
    Ch0: MRL=0.026, PPC=-0.007, p=0.9147, n=136
    Ch1: MRL=0.059, PPC=-0.004, p=0.6254, n=136
    Ch2: MRL=0.037, PPC=-0.006, p=0.8315, n=136
    Ch3: MRL=0.057, 

\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_gui.py:506: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.result_fig.tight_layout()
\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_gui.py:506: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.result_fig.tight_layout()
\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_gui.py:506: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.result_fig.tight_layout()
\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_gui.py:506: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.result_fig.tight_layout()
\\nagoya_amcbnas2\wakela

  プロットエラー: Dimensions of C (15, 204) should be one smaller than X(204) and Y(16) while using shading='flat' see help(pcolormesh)
  CSV保存: \\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\0209解析\260114-4_Wildtype_S1BF_920_wisker_phase_locking.csv


Traceback (most recent call last):
  File "\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_plotting_v6.py", line 649, in plot_phase_grand_summary
    _plot_page1(fig1, lfp_cleaned, lfp_times, fs, stim_times,
  File "\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_plotting_v6.py", line 96, in _plot_page1
    im = ax_psd.pcolormesh(freqs, np.arange(n_ch + 1) - 0.5, psd_matrix.T,
  File "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\matplotlib\__init__.py", line 1524, in inner
    return func(
  File "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\matplotlib\axes\_axes.py", line 6528, in pcolormesh
    X, Y, C, shading = self._pcolorargs('pcolormesh', *args,
  File "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\matplotlib\axes\_axes.py", line 6060, in _pcolorargs
    raise TypeError(f"Dimensions of C {C.shape} should"
TypeError: Dimensions of C (15, 204) should be one smaller than X(204) 

  プロットエラー: Dimensions of C (15, 204) should be one smaller than X(204) and Y(16) while using shading='flat' see help(pcolormesh)
  CSV保存: \\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\0209解析\260114-4_Wildtype_S1BF_920_wisker_phase_locking.csv


Traceback (most recent call last):
  File "\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_plotting_v6.py", line 649, in plot_phase_grand_summary
    _plot_page1(fig1, lfp_cleaned, lfp_times, fs, stim_times,
  File "\\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\phase_plotting_v6.py", line 96, in _plot_page1
    im = ax_psd.pcolormesh(freqs, np.arange(n_ch + 1) - 0.5, psd_matrix.T,
  File "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\matplotlib\__init__.py", line 1524, in inner
    return func(
  File "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\matplotlib\axes\_axes.py", line 6528, in pcolormesh
    X, Y, C, shading = self._pcolorargs('pcolormesh', *args,
  File "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\matplotlib\axes\_axes.py", line 6060, in _pcolorargs
    raise TypeError(f"Dimensions of C {C.shape} should"
TypeError: Dimensions of C (15, 204) should be one smaller than X(204) 

KeyboardInterrupt: 

## 5. 完了

In [ ]:
print("=" * 60)
print("  Neuronexus Analysis v6 完了!")
print(f"  出力先: {output_dir}")
print("=" * 60)

print("\n--- 結果サマリー ---")
if lfp_results is not None:
    n_ch = lfp_results.get('lfp_cleaned', np.empty((0,0))).shape[1] if lfp_results.get('lfp_cleaned') is not None else 0
    print(f"  LFP: {n_ch} チャンネル処理済み")
else:
    print("  LFP: 未処理")

if spike_results is not None:
    n_units = sum(len(r.units) for r in spike_results.values())
    print(f"  スパイク: {n_units} units ({len(spike_results)} チャンネル)")
else:
    print("  スパイク: 未処理")

if phase_results is not None:
    print(f"  位相ロック: 解析完了")
else:
    print("  位相ロック: 未処理")2

SyntaxError: invalid syntax (3759406776.py, line 22)

In [ ]:
  import sys
  print("Python:", sys.executable)

  # KiloSort直接テスト
  try:
      import kilosort
      print("KiloSort:", kilosort.__version__)
  except Exception as e:
      print("KiloSort import ERROR:", e)

  # PyTorch/CUDAテスト
  try:
      import torch
      print("PyTorch:", torch.__version__)
      print("CUDA:", torch.cuda.is_available())
  except Exception as e:
      print("PyTorch import ERROR:", e)

  # wrapper テスト
  try:
      from kilosort_wrapper import is_kilosort_available
      print("is_kilosort_available():", is_kilosort_available())
  except Exception as e:
      print("wrapper ERROR:", e)

Python: c:\Users\wakel\anaconda3\envs\neoeeg\python.exe
KiloSort import ERROR: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.
PyTorch import ERROR: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.
is_kilosort_available(): False
